In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, Dropout,Activation, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping
import tensorflow as tf
from PIL import Image 
import os
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
#from tensorflow.keras.applications.VGG import preprocess_input

from tensorflow.keras.optimizers import Adam

## Loading Dataset

In [ ]:

train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train['label'] = train['label'].astype('string')
train.head()

In [ ]:
names_of_disease = pd.read_json('../input/cassava-leaf-disease-classification/label_num_to_disease_map.json', typ='series')
names_of_disease

In [ ]:
plt.figure(figsize=(17, 12))
for i in range(8):
    plt.subplot(4, 4, i + 1)
    image = Image.open('../input/cassava-leaf-disease-classification/train_images/'+ train.iloc[i]['image_id'])
    array = np.array(image)
    plt.imshow(array)
    label=train.iloc[i]['label']
    plt.title(f'{names_of_disease[int(label)]}')
plt.show()

In [ ]:
sizes = []
for i in range(1, len(train), 250):
    image = Image.open('../input/cassava-leaf-disease-classification/train_images/' + train.iloc[i]['image_id'])
    array = np.array(image)
    sizes.append(array.shape)
print('Picture size', set(sizes))

## Data Augmentation

### data splitting 0.2

In [ ]:
datagen = ImageDataGenerator(validation_split=0.2,
                             vertical_flip=True,
                             rotation_range=90,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             horizontal_flip=True)
train_datagen_flow = datagen.flow_from_dataframe(
    dataframe=train,
    directory='../input/cassava-leaf-disease-classification/train_images',
    x_col='image_id',
    y_col='label',
    target_size=(300, 300),
    batch_size=20,
    subset='training',
    seed=0)


In [ ]:
valid_datagen_flow = datagen.flow_from_dataframe(
    dataframe=train,
    directory='../input/cassava-leaf-disease-classification/train_images',
    x_col='image_id',
    y_col='label',
    target_size=(300, 300),
    batch_size=20,    
    subset='validation',
    seed=0)


## Training with vgg16

### Optimizer: Adam

### learning rate=0.0002

In [ ]:
import h5py
model=Sequential()

optimizer = Adam(lr=0.0002)
backbone = VGG16(weights='../input/vgg16-weights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False,  pooling='max')

                        
model.add(backbone)
model.add(Dense(5, activation='softmax'))
model.compile(loss="categorical_crossentropy", 
              optimizer=optimizer, 
              metrics=["accuracy"])
model.summary()

In [ ]:

model.fit_generator(train_datagen_flow,
                    validation_data=valid_datagen_flow, 
                    epochs=10, 
                    verbose=1,shuffle=False)

## Submission

In [ ]:
submission = pd.DataFrame(columns=['image_id','label'])
for image_name in os.listdir('../input/cassava-leaf-disease-classification/train_images'):
    image_path = os.path.join('../input/cassava-leaf-disease-classification/train_images', image_name)
    image = tf.keras.preprocessing.image.load_img(image_path)
    resized_image = image.resize((300,300))
    numpied_image = np.expand_dims(resized_image, 0)
    tensored_image = tf.cast(numpied_image, tf.float32)
    submission = submission.append(pd.DataFrame({'image_id': image_name,
                                                 'label': model.predict_classes(tensored_image)}))


In [ ]:
submission.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
submission.head()